# Importing modules and mounting Drive

In [0]:
%tensorflow_version 2.x

In [0]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/Colab Notebooks/image-captioning/'

Mounted at /content/drive


In [0]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('seaborn-darkgrid')

from tensorflow import keras
from nltk.translate.bleu_score import corpus_bleu

import re
import random

import joblib
from tqdm import tqdm
import glob
from IPython.display import clear_output

# Preparing Data

## Captions

In [0]:
DATA_TEXT = PATH + 'Flickr8k/Flickr8k_text/'

In [0]:
images_and_captions = {}

with open(DATA_TEXT + 'Flickr8k.token.txt', 'r') as f:
    # each line contains the image id and one of its captions
    caption = f.readline()
    while caption:
        tokens = caption.split()

        # separate image id and image caption
        image_id, image_caption = tokens[0], tokens[1:]
        # keep only the id
        image_id = image_id.split('.')[0]
        
        image_caption = ' '.join(image_caption)
        
        # add the caption to the dictionary of ids
        if image_id not in images_and_captions:
            images_and_captions[image_id] = []
        images_and_captions[image_id].append(image_caption)

        caption = f.readline()

In [0]:
len(images_and_captions)

8092

In [0]:
def clean_text(text):
    """
    Normalizes and cleans text from punctuation and numbers, and add the tags
    for start and end of text
    :param text: string with text to be cleaned and added tags
    :return : string prepared text
    """
    text = text.lower()
    text = re.sub('[^a-zA-Z ]+', '', text).split()
    text = '<start> ' + ' '.join(text) + ' <end>'
    return text

In [0]:
# normalizes all the captions
for image, captions in images_and_captions.items():
    cleaned_text = []
    for caption in captions:
        cleaned_text.append(clean_text(caption))
    images_and_captions[image] = cleaned_text

In [0]:
# save all the captions in a list for vocabulary selection
all_captions = []
for captions in images_and_captions.values():
    for caption in captions:
        all_captions.append(caption)

In [0]:
len(all_captions)

40460

In [0]:
def count_vocabulary(all_captions):
    """
    Counts the size of the vocabulary
    :param all_captions: list with all captions
    :return : integer size of the vocabulary
    """
    vocab = set()
    for caption in all_captions:
        [vocab.add(word) for word in caption.split()]
    return len(vocab)

In [0]:
count_vocabulary(all_captions)

8780

In [0]:
# we will keep only the words in our vocabulary that apears more than threshold times
# and with size bigger than min_size
threshold = 15
min_size = 1

word_counts = {}

for caption in all_captions:
    for word in caption.split(' '):
        word_counts[word] = word_counts.get(word, 0) + 1

vocabulary = [word for word in word_counts if (word_counts[word] >= threshold and len(word) > min_size)]

len(vocabulary)

1554

In [0]:
def reduce_vocabulary(captions, vocabulary):
    """
    Keep only the words in our vocabulary
    :param captions: string with caption
    :param vocabulary: list with the vocabulary
    :return : string new caption with the selected vocabulary
    """
    new_captions = []
    for caption in captions:
        c = [word for word in caption.split() if word in vocabulary]
        c = ' '.join(c)
        new_captions.append(c)
    return new_captions

In [0]:
# now we modifie our captions to keep only the words in our vocabulary
for image, captions in images_and_captions.items():
    images_and_captions[image] = reduce_vocabulary(captions, vocabulary)

In [0]:
all_captions = reduce_vocabulary(all_captions, vocabulary)
count_vocabulary(all_captions)

In [0]:
# build to dictionaries for the vocabulary tokenization

# maps each word with an integer
word_to_index = {}
# maps each integer with a word
index_to_word = {}

for i, word in enumerate(vocabulary):
    word_to_index[word] = i + 1
    index_to_word[i + 1] = word

In [0]:
joblib.dump(word_to_index, PATH + 'word_to_index.dump')
joblib.dump(index_to_word, PATH + 'index_to_word.dump')

In [0]:
# we now get the lenght of the biggest caption and the size ou our vocabulary
MAX_LENGTH = max([len(caption.split()) for caption in all_captions])
VOCAB_SIZE = len(vocabulary) + 1

MAX_LENGTH, VOCAB_SIZE

(31, 1555)

In [0]:
# we will use the GloVe pretrained model to define the weights of the embedding
# layer on out model
EMBEDDING_DIM = 200

embedding_vector = {}

with open(PATH + 'models/glove.6B.200d.txt', 'r') as f:
    for line in f:
        value = line.split(' ')
        word = value[0]
        coef = np.array(value[1:], dtype = 'float32')
        embedding_vector[word] = coef

In [0]:
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

for word, i in word_to_index.items():
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

## Images

In [0]:
DATA_IMAGES = PATH + 'Flickr8k/Flickr8k_Dataset/Flicker8k_Dataset/'

In [0]:
# we use the pretrained VGG19 to extract the features of the images
FEATURES_DIM = 4096

vgg19 = keras.applications.VGG19()
vgg19 = keras.models.Model(inputs=vgg19.inputs, outputs=vgg19.layers[-2].output)

In [0]:
image_paths = glob.glob(DATA_IMAGES + '*.jpg')

In [0]:
len(image_paths)

8092

In [0]:
def load_image(image_path):
    """
    Processes and image for the VGG19 model
    :param image_path: string with the path to the image
    :img : numpy array with the preprocessd image to be fed to the VGG19 model
    """
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = keras.preprocessing.image.img_to_array(img)
    img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
    img = keras.applications.vgg19.preprocess_input(img)
    return img

In [0]:
# the features dictionary will save tje features extracted from the VGG19 model
# for each image
features = {}

for image in tqdm(image_paths):
    img = load_image(image)
    feature = vgg19.predict(img, verbose=0)
    image_id = image.split('/')[-1].split('.')[0]
    features[image_id] = feature

100%|██████████| 8092/8092 [05:43<00:00, 23.58it/s]


In [0]:
# the images ids for the images_and_captions and features dictionaries differ
set(images_and_captions.keys()) - set(features.keys()), set(features.keys()) - set(images_and_captions.keys())

({'2258277193_586949ec62'}, {'3339586622_a7676b30e1 (1)'})

In [0]:
# remove the differing images
images.pop('2258277193_586949ec62'), features.pop('3339586622_a7676b30e1 (1)');

# Creating the dataset

The Flickr8k dataset has the images separated in training, validation and test.

## Get training images

In [0]:
train_images = set()

with open(DATA_TEXT + 'Flickr_8k.trainImages.txt', 'r') as f:
    image = f.readline()
    while image:
        image_id = image.split('.')[0]
        
        train_images.add(image_id)

        image = f.readline()

## Get validation images

In [0]:
valid_images = set()

with open(DATA_TEXT + 'Flickr_8k.devImages.txt', 'r') as f:
    image = f.readline()
    while image:
        image_id = image.split('.')[0]
        
        valid_images.add(image_id)

        image = f.readline()

## Get test images

In [0]:
test_images = set()

with open(DATA_TEXT + 'Flickr_8k.testImages.txt', 'r') as f:
    image = f.readline()
    while image:
        image_id = image.split('.')[0]
        
        test_images.add(image_id)

        image = f.readline()

## Removing missing images

In [0]:
# the tokens file has images that are missing on the separated sets
missing_images = set(features.keys()) - (train_images | valid_images | test_images)
len(missing_images)

91

In [0]:
# we remove these missing images from our 
# features and images_and_captions dictionaries
for image in missing_images:
    features.pop(image)
    images_and_captions.pop(image)

## Separating train, validation and test sets

In [0]:
def get_images_captions(captions, dataset):
    """
    Separates the captions for the dataset
    :param captions: dictinary with the captions for each image
    :param dataset: set with the image ids for the dataset
    :return : dictionary with the captions for the image ids in dataset
    """
    images_captions = {}

    for image in dataset:
        images_captions[image] = captions[image]
    
    return images_captions

def get_images_features(features, dataset):
    """
    Separates the features for the dataset
    :param features: dictinary with the features for each image
    :param dataset: set with the image ids for the dataset
    :return : dictionary with the features for the image ids in dataset
    """
    images_features = {}

    for image in dataset:
        images_features[image] = features[image]
    
    return images_features

In [0]:
train = get_images_captions(images_and_captions, train_images), get_images_features(features, train_images)
valid = get_images_captions(images_and_captions, valid_images), get_images_features(features, valid_images)
test = get_images_captions(images_and_captions, test_images), get_images_features(features, test_images)

# Creting sequences vectors generators

The data to be fed to the model contains a tuple of two with:
 * a list containg the features extracted from the VGG19 model and a list containg a text sequence
 * the next word to be added to the sequence

This makes that our data could have **N_IMAGES** x **MAX_LENGTH** x **N_CAPTIONS** rows

Each row will have **FEATURES_DIM** + **MAX_LENGTH** values

Each text sequence will be put in a bigger hyperspace from our embedding layer

To resume, the data can't fit in memory, so we will use a generetor

In [0]:
def create_sequences_vectors_generator(word_to_index, max_length, images_and_captions, features, vocab_size, batch_size):
    """
    Generator that yelds inputs and output for the model in batches
    :param word_to_index: dictionary with the vocabulary
    :param max_length: integer with the length of the biggest string
    :param images_and_captions: dictionary with the captions for each image id
    :param features: dictionary with the features for each image id
    :param vocab_size: integer with the size of the vocabulary
    :param batch_size: integer with size of the batch for the generator
    :yeld : tuple of two with a list containing features and sequences, and the next word
    """
    F, S, y = [], [], []
    batch = 0
    while True:
        for image, captions in images.items():
            batch += 1
            
            for caption in captions:
                sequence = [word_to_index[word] for word in caption.split(' ') if word in word_to_index]
            
                for i in range(1, len(sequence)):
                    in_sequence, out_sequence = sequence[:i], sequence[i]
                    in_sequence = keras.preprocessing.sequence.pad_sequences([in_sequence], maxlen=max_length)[0]
                    out_sequence = keras.utils.to_categorical([out_sequence], num_classes=vocab_size)[0]
                    F.append(features[image][0])
                    S.append(in_sequence)
                    y.append(out_sequence)
            if batch == batch_size:
                yield ([np.array(F), np.array(S)], np.array(y))
                F, S, y = [], [], []
                batch = 0

In [0]:
# define the batch size to the generator
BATCH_SIZE = 64
# define the number of steps the model will take each epoch
STEPS_PER_EPOCH = len(train[0]) // BATCH_SIZE
VALIDATION_STEPS = len(valid[0]) // BATCH_SIZE

train_generator = create_sequences_vectors_generator(word_to_index, MAX_LENGTH, train[0], train[1], VOCAB_SIZE, BATCH_SIZE)
valid_generator = create_sequences_vectors_generator(word_to_index, MAX_LENGTH, valid[0], valid[1], VOCAB_SIZE, BATCH_SIZE)

# Training model

In [0]:
def build_model(vocab_size, max_length, features_dim, embedding_dim, version):
    """
    :param vocab_size: integer with the size of the vocabulary
    :param max_length: integer with the length of the biggest sequence
    :param features_dim: integer with the size of the features
    :param embedding_dim: integer with the size of the embedding
    :param version: integer with the version of the model
    :return : keras model with 2 inputs, 
        one for the features and one for the sequences,
        and one output, the next word for the sequence
    """
    # Features model
    input1 = keras.layers.Input(shape=(features_dim,))
    features1 = keras.layers.Dropout(0.5)(input1)
    features2 = keras.layers.Dense(256, activation='relu')(features1)

    # Sequences model
    input2 = keras.layers.Input(shape=(max_length,))
    sequence1 = keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable = False, mask_zero=True)(input2)
    sequence2 = keras.layers.Dropout(0.5)(sequence1)
    sequence3 = keras.layers.LSTM(256)(sequence2)

    # Prediction model
    predicion1 = keras.layers.Add()([features2, sequence3])
    predicion2 = keras.layers.Dense(256, activation='relu')(predicion1)
    output = keras.layers.Dense(vocab_size, activation='softmax')(predicion2)

    # Model with [features, sequences] input, [next word]  output
    model = keras.models.Model(inputs=[input1, input2], outputs=output, name='model-v{}'.format(version))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    print(model.summary())
    keras.utils.plot_model(model, to_file=PATH + '/misc/{}.png'.format(model.name), show_shapes=True);
    return model

In [0]:
# creates the model
model = define_model(VOCAB_SIZE, MAX_LENGTH, FEATURES_DIM, EMBEDDING_DIM, 3)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 31)]         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            [(None, 4096)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 31, 200)      311000      input_10[0][0]                   
__________________________________________________________________________________________________
dropout (Dropout)               (None, 4096)         0           input_9[0][0]                    
____________________________________________________________________________________________

In [0]:
# creates a checkpoint for each improving epoch
checkpoint_path = PATH + 'models/' + model.name + '/ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=0, save_best_only=True)

# creates a checkpoint for early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# creates a checkpoint for logging the loss at each epoch
logger_path = PATH + 'misc/training-{}.csv'.format(model.name)
logger = keras.callbacks.CSVLogger(logger_path, separator=',', append=False)

In [0]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=STEPS_PER_EPOCH, 
                              epochs=200, callbacks=[checkpoint, early_stopping, logger],
                              validation_data=valid_generator, validation_steps=VALIDATION_STEPS)

Epoch 1/200
93/93 [==============================] - 58s 626ms/step - loss: 5.0610 - val_loss: 4.3744
Epoch 2/200
93/93 [==============================] - 56s 604ms/step - loss: 4.0693 - val_loss: 3.8597
Epoch 3/200
93/93 [==============================] - 57s 613ms/step - loss: 3.6902 - val_loss: 3.6596
Epoch 4/200
93/93 [==============================] - 56s 602ms/step - loss: 3.4731 - val_loss: 3.5477
Epoch 5/200
93/93 [==============================] - 57s 610ms/step - loss: 3.3135 - val_loss: 3.4632
Epoch 6/200
93/93 [==============================] - 57s 608ms/step - loss: 3.1974 - val_loss: 3.4113
Epoch 7/200
93/93 [==============================] - 56s 604ms/step - loss: 3.0994 - val_loss: 3.3884
Epoch 8/200
93/93 [==============================] - 57s 608ms/step - loss: 3.0200 - val_loss: 3.3665
Epoch 9/200
93/93 [==============================] - 56s 607ms/step - loss: 2.9537 - val_loss: 3.3578
Epoch 10/200
93/93 [==============================] - 56s 606ms/step - loss: 2.893

In [0]:
model.save(PATH + 'models/{}.h5'.format(model.name))

# Testing model

In [0]:
def generate_caption(model, word_to_index, index_to_word, features, max_length):
    """
    Generates a caption fot the image using a greedy aproach
    :param model: keras model to predict the caption
    :param word_to_index: dictionary with vocabulary mapping string to integer
    :param index_to_word: dictionary with vocabulary mapping integer to string
    :param features: array with VGG19 features for the image
    :param max_length: integer with maximum length for a caption
    :return : string with caption
    """
    caption = '<start>'
    
    for i in range(max_length):
        
        sequence = [word_to_index[word] for word in caption.split(' ') if word in word_to_index]
        
        sequence = keras.preprocessing.sequence.pad_sequences([sequence], maxlen=max_length)

        next_word = model.predict(([[features[0]],[sequence[0]]]), verbose=0)
        
        next_word = np.argmax(next_word)
        
        word = index_to_word[next_word]
        
        if word is None:
            word += ' <end>'
            break
        
        caption += ' ' + word
        
        if word == '<end>':
            break
            
    caption = caption.split(' ')
    caption = caption[1:-1]
    caption = ' '.join(caption)
    return caption

In [0]:
def test_model(model, word_to_index, index_to_word, test_images, test_features, test_captions):
    """
    Selects and image, generates a caption, saves and shows it
    :param model: keras model to predict the caption
    :param word_to_index: dictionary with vocabulary mapping string to integer
    :param index_to_word: dictionary with vocabulary mapping integer to string
    :param test_images: set with ids for images to be tested
    :param test_features: dictionaty with VGG19 features for each image
    :param test_captions: dictionaty with captions for each image
    """
    sample = random.sample(set(test_images), 1)[0]

    path = DATA_IMAGES + sample + '.jpg'

    plt.figure(figsize=(10,10))
    
    im = plt.imread(path)
    plt.imshow(im)

    plt.title( generate_caption(model, word_to_index, index_to_word, test_features[sample], MAX_LENGTH) )
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    
    plt.savefig(PATH + '/predictions/' + '{}_{}.png'.format(sample, model.name))
    plt.show()

In [0]:
for i in range(50):
    clear_output(wait=True)
    test_model(model, word_to_index, index_to_word, test_images, test[1], test[0])

# Evalutate model

In [0]:
def evaluate_model(model, images_and_captions, features, word_to_index, index_to_word, max_length):
	"""
	Evaluates the model according to the BLEU metric, wich evaluates
	the quality of text which has been machine-translated 
	from one natural language to another.
	:param model: keras model to be evaluated
	:param images_and_captions: dictionary with captions for each image
	:param features: dictionary with features for each image
	:param word_to_index: dictionary with vocabulary mapping string to integer
    :param index_to_word: dictionary with vocabulary mapping integer to string
    :param max_length: integer with maximum length for a caption
	"""
	actual, predicted = [], []

	for image, captions in tqdm(images_and_captions.items()):
		
		y_pred = generate_caption(model, word_to_index, index_to_word, features[image], max_length)
		
		y_true = [caption.split() for caption in captions]
		
		actual.append(y_true)
		predicted.append(y_pred.split())

	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [0]:
# evaluate_model(model, test[0], test[1], word_to_index, index_to_word, MAX_LENGTH)

100%|██████████| 1000/1000 [6:22:52<00:00, 18.16s/it]


BLEU-1: 0.454665
BLEU-2: 0.277141
BLEU-3: 0.196413
BLEU-4: 0.094753
